# Defi IA 2023 - Prédiction du prix d'une chambre d'hôtels

In [6]:
import pandas as pd
import numpy as np
import os
import glob
import urllib.parse
import requests
import os

In [7]:
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)
user_id = '88be3640-82ae-46c6-ac59-92a9679d453d'

## Création d'un avatar

In [ ]:
name = 'laulau' #donner le nom de l'avatar
new_avatar = requests.post(path(f'avatars/{user_id}/{name}'))

In [8]:
avatar_info = requests.get(path(f"avatars/{user_id}"))
avatar_names = []

for avatar in avatar_info.json():
    avatar_names.append(avatar['name'])

avatar_names

['laulau', 'Nini', 'Tess']

## Excécution des requêtes

In [9]:
cities = ['amsterdam','madrid','paris','rome','sofia','vienna', 'valletta', 'vilnius', 'copenhagen', 'sofia']
dates = [5,3,2,1]
languages = ['belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'luxembourgish', 'portuguese', 'romanian' ]
mobiles = [0,1]
avatar = 'Tess' #nom de l'avatar à tester

if not os.path.exists('requests'):
    os.mkdir('requests')


#listes contenant toutes les combinaisons possibles pour un avatar
combination=[]
for count1,date in enumerate(dates):
    for count2,city in enumerate(cities):
        for count3,language in enumerate(languages):
            for count4,mobile in enumerate(mobiles):
                combination.append((city,date,language,mobile))
                params = {
                "avatar_name": avatar,
                "language": language,
                "city": city,
                "date": date,
                "mobile": mobile,}

                ## REQUÊTE ##
                requete = requests.get(path(f"pricing/{user_id}"), params=params)
                requete.json()
                pricing_requests = []

                request = [requete]
                for requete in request:
                    pricing_requests.append(
                        pd.DataFrame(requete.json()['prices']).assign(**requete.json()['request'])
                    )

                pricing_requests = pd.concat(pricing_requests)
                pricing_requests.to_csv(os.path.join('requests', str(avatar) +'_'+str(city)+ '_' + str(date) + '_' + str(language) + '_' + str(mobile)+'.csv'))

In [10]:
requete.json()

{'prices': [{'hotel_id': 141, 'price': 89, 'stock': 0},
  {'hotel_id': 109, 'price': 95, 'stock': 1},
  {'hotel_id': 764, 'price': 67, 'stock': 0},
  {'hotel_id': 472, 'price': 46, 'stock': 0},
  {'hotel_id': 201, 'price': 89, 'stock': 2},
  {'hotel_id': 578, 'price': 110, 'stock': 12},
  {'hotel_id': 272, 'price': 49, 'stock': 0},
  {'hotel_id': 665, 'price': 64, 'stock': 7},
  {'hotel_id': 608, 'price': 96, 'stock': 2},
  {'hotel_id': 590, 'price': 87, 'stock': 0},
  {'hotel_id': 4, 'price': 44, 'stock': 0},
  {'hotel_id': 544, 'price': 66, 'stock': 0},
  {'hotel_id': 553, 'price': 101, 'stock': 0},
  {'hotel_id': 542, 'price': 66, 'stock': 0},
  {'hotel_id': 815, 'price': 248, 'stock': 1},
  {'hotel_id': 637, 'price': 65, 'stock': 0},
  {'hotel_id': 574, 'price': 65, 'stock': 0},
  {'hotel_id': 899, 'price': 94, 'stock': 1},
  {'hotel_id': 683, 'price': 89, 'stock': 0},
  {'hotel_id': 304, 'price': 83, 'stock': 6},
  {'hotel_id': 145, 'price': 95, 'stock': 1},
  {'hotel_id': 107, 'p

## Mise en lien des requêtes et des features des differents hotels

In [4]:
hotels_features = pd.read_csv(os.path.join('features', 'features_hotels.csv'))
laulau_requests_copenhagen = pd.read_csv(os.path.join('requests', 'laulau_copenhagen_30_0.csv'))
laulau_requests_copenhagen = laulau_requests_copenhagen.drop(columns = 'Unnamed: 0')

In [5]:
laulau_requests_copenhagen

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,519,208,68,copenhagen,30,french,0,3533
1,523,85,41,copenhagen,30,french,0,3533
2,898,86,24,copenhagen,30,french,0,3533
3,770,210,73,copenhagen,30,french,0,3533
4,562,215,74,copenhagen,30,french,0,3533
...,...,...,...,...,...,...,...,...
139,332,208,86,copenhagen,30,french,0,3533
140,978,299,120,copenhagen,30,french,0,3533
141,335,91,46,copenhagen,30,french,0,3533
142,385,283,101,copenhagen,30,french,0,3533


La commande ci-dessous permet de charger les informations en lien avec la requête lancée. Ainsi, on peut lier les caractéristiques de l'hôtel avec la recherche effectuée par un avatar, dans une ville et une langue particulière.  

In [6]:
features_laulau_copenhagen = hotels_features.loc[hotels_features['hotel_id'].loc[laulau_requests_copenhagen['hotel_id']]]
features_laulau_copenhagen = features_laulau_copenhagen.reset_index()

In [7]:
df = pd.concat([features_laulau_copenhagen, laulau_requests_copenhagen[['price', 'stock', 'date', 'language', 'mobile']]], axis = 1)
df

,index,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile
0,519,519,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0
1,523,523,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0
2,898,898,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0
3,770,770,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0
4,562,562,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,332,332,Chillton Worldwide,Quadrupletree,copenhagen,1,1,0,208,86,30,french,0
140,978,978,Chillton Worldwide,Chill Garden Inn,copenhagen,1,0,0,299,120,30,french,0
141,335,335,Independant,Independant,copenhagen,0,0,0,91,46,30,french,0
142,385,385,Accar Hotels,Safitel,copenhagen,0,0,0,283,101,30,french,0


## Automatisation du code pour création des datasets

On va chercher à créer une liste contenant le nom de tous les avatars.

In [11]:
r = requests.get(path(f"avatars/{user_id}"))
list_avatars = []
for avatar in r.json():
    list_avatars.append(avatar['name'])

Ayant tous les avatars, on peut créer les datasets pour chacun des avatars, s'il n'existe pas déjà, et les compléter avec les données récupérées.

In [12]:
hotels_features = pd.read_csv(os.path.join('features', 'features_hotels.csv'))

Le code ci-dessous permet de créer le dataset associé à un avatar précis. 

In [13]:
list_requests = glob.glob(os.path.join('requests', '*.csv'))
list_requests = [request.split('\\')[-1] for request in list_requests]
df = pd.DataFrame([])

if not os.path.exists('datasets'):
    os.mkdir('datasets')

i= 0
for avatar in ['Tess']:
    for request in list_requests:
        name_avatar_request = request.split('_')[0] 
        if (avatar==name_avatar_request):
            print(i)

            if not os.path.exists(os.path.join('datasets', avatar + '_data.csv')):
                request_avatar = pd.read_csv(os.path.join('requests', request))
                features_avatar = hotels_features.loc[hotels_features['hotel_id'].loc[request_avatar['hotel_id']]]
                features_avatar = features_avatar.reset_index()
                df = pd.concat([features_avatar, request_avatar[['price', 'stock', 'date', 'language', 'mobile']]], axis = 1)
                df['avatar_name'] = [avatar for i in range(len(df.index))]
                df = df.drop(columns = ['index'])
                df.to_csv(os.path.join('datasets', avatar + '_data.csv'), index=False)
            else:
                df_old = pd.read_csv(os.path.join('datasets', avatar + '_data.csv'))
                request_avatar = pd.read_csv(os.path.join('requests', request))
                features_avatar = hotels_features.loc[hotels_features['hotel_id'].loc[request_avatar['hotel_id']]]
                features_avatar = features_avatar.reset_index()
                df_aux = pd.concat([features_avatar, request_avatar[['price', 'stock', 'date', 'language', 'mobile']]], axis = 1)
                df = pd.concat([df,df_aux], axis = 0)
                df = df.drop(columns = ['index'])

                df['avatar_name'] = [avatar for i in range(len(df.index))]
                df.to_csv(os.path.join('datasets', avatar + '_data.csv'), index=False)
            i+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
#Concaténatation des datasets de chacun des avatars pour créer un dataset global
datasets = glob.glob(os.path.join('datasets', '*.csv'))

for dataset_path in datasets:
    dataset = pd.read_csv(dataset_path)
    if not os.path.exists('data.csv'):
        df = pd.DataFrame([])
        df = dataset
        df.to_csv('data.csv', index= False)
    else:
        df = pd.read_csv('data.csv')
        print(df.head())
        df = pd.concat([df, dataset], axis = 0)
        df.to_csv('data.csv', index=False)

   hotel_id                   group         brand       city  parking  pool  \
0       917  Morriott International       Corlton  amsterdam        1     1   
1       101            Accar Hotels       Safitel  amsterdam        1     0   
2       762                Yin Yang      Ardisson  amsterdam        1     1   
3       140            Boss Western  Boss Western  amsterdam        0     0   
4       256            Accar Hotels          Ibas  amsterdam        1     0   

   children_policy  price  stock  date  language  mobile avatar_name  
0                0    366     14    10  austrian       0      laulau  
1                0    359     10    10  austrian       0      laulau  
2                0    193     10    10  austrian       0      laulau  
3                0    205      4    10  austrian       0      laulau  
4                0     86      0    10  austrian       0      laulau  
   hotel_id                   group         brand       city  parking  pool  \
0       917  Morriot

In [27]:
df.head()

,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name
0,917,Morriott International,Corlton,amsterdam,1,1,0,366,14,10,austrian,0,laulau
1,101,Accar Hotels,Safitel,amsterdam,1,0,0,359,10,10,austrian,0,laulau
2,762,Yin Yang,Ardisson,amsterdam,1,1,0,193,10,10,austrian,0,laulau
3,140,Boss Western,Boss Western,amsterdam,0,0,0,205,4,10,austrian,0,laulau
4,256,Accar Hotels,Ibas,amsterdam,1,0,0,86,0,10,austrian,0,laulau


In [15]:
pd.read_csv(os.path.join('datasets', 'laulau_data.csv'))

,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name
0,917,Morriott International,Corlton,amsterdam,1,1,0,366,14,10,austrian,0,laulau
1,101,Accar Hotels,Safitel,amsterdam,1,0,0,359,10,10,austrian,0,laulau
2,762,Yin Yang,Ardisson,amsterdam,1,1,0,193,10,10,austrian,0,laulau
3,140,Boss Western,Boss Western,amsterdam,0,0,0,205,4,10,austrian,0,laulau
4,256,Accar Hotels,Ibas,amsterdam,1,0,0,86,0,10,austrian,0,laulau
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98711,498,Boss Western,Boss Western,vilnius,0,0,0,94,40,30,czech,1,laulau
98712,516,Morriott International,Corlton,vilnius,1,0,2,195,117,30,czech,1,laulau
98713,187,Accar Hotels,Marcure,vilnius,1,1,0,137,86,30,czech,1,laulau
98714,687,Chillton Worldwide,Tripletree,vilnius,0,0,0,110,52,30,czech,1,laulau
